In [1]:
#import necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#load webpage https://www.theguardian.com/football
# headers will be used to trick the website that we open it like a browser, not like a scrapping tool. 
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
page = 'https://www.theguardian.com/football/premierleague/table'
try:
    content = requests.get(page, headers=headers)
except:
    print('Source 1 unavailable. Check the internet connection')
soup = BeautifulSoup(content.content, 'html.parser')
#identify columns from content page
player_table = soup.find("table")
columns = []
for th in player_table.find_all('th'):
    title = th.text
    columns.append(title)
data_tg = pd.DataFrame(columns = columns)
leagues = ['premierleague', 'bundesligafootball', 'serieafootball', 'laligafootball', 'ligue1football']
for league in leagues:
    page = 'https://www.theguardian.com/football/'+ league + '/table'
    try:
        content = requests.get(page, headers=headers)
    except:
        print('Source 1 unavailable. Check the internet connection')
    soup = BeautifulSoup(content.content, 'html.parser')
    #identify columns from content page
    player_table = soup.find("table")
    listo = []
    for tr in player_table.find_all('tr'):
            data = tr.find_all('td')
            for tr in data:
                item = tr.text
                listo.append(item.strip())
    for i in range(0, len(listo), len(columns)):
        kalisto = listo[i:i + len(columns)]            
        data_tg_length = len(data_tg)
        data_tg.loc[data_tg_length] = kalisto


In [3]:
# make copy of data_frame
df_source3 = data_tg.copy()

In [4]:
# drop unnecessary columns and change type to numeric continuous variables
# create a new variable avg_points 
df_source3 = df_source3.drop(['P','Form'], axis=1)
df_source3[['GP', 'W', 'D', 'L','F', 'A', 'GD', 'Pts']] = df_source3[['GP', 'W', 'D', 'L','F', 'A', 'GD', 'Pts']].apply(pd.to_numeric) 
df_source3['club_avg_points']=df_source3['Pts'] / df_source3['GP']
df_source3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Team             98 non-null     object 
 1   GP               98 non-null     int64  
 2   W                98 non-null     int64  
 3   D                98 non-null     int64  
 4   L                98 non-null     int64  
 5   F                98 non-null     int64  
 6   A                98 non-null     int64  
 7   GD               98 non-null     int64  
 8   Pts              98 non-null     int64  
 9   club_avg_points  98 non-null     float64
dtypes: float64(1), int64(8), object(1)
memory usage: 8.4+ KB


In [5]:
# rename a column
df_source3.rename(columns={'Team': 'club'}, inplace = True)

In [8]:
# checking our DataFrame
df_source3

,club,GP,W,D,L,F,A,GD,Pts,club_avg_points
0,Chelsea,13,9,3,1,31,5,26,30,2.307692
1,Man City,13,9,2,2,27,7,20,29,2.230769
2,Liverpool,13,8,4,1,39,11,28,28,2.153846
3,West Ham,13,7,2,4,24,16,8,23,1.769231
4,Arsenal,13,7,2,4,15,17,-2,23,1.769231
...,...,...,...,...,...,...,...,...,...,...
93,Bordeaux,15,2,7,6,22,32,-10,13,0.866667
94,Troyes,15,3,4,8,13,23,-10,13,0.866667
95,Clermont Foot,15,3,4,8,17,29,-12,13,0.866667
96,Metz,15,2,6,7,17,30,-13,12,0.800000


In [9]:
# export our data into csv file.
df_source3.to_csv('../data/df_source3.csv', index=False)